# Data Preprocessing
### Load raw data and preprocess data

### 1. Importing libraries

In [1]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
import re
import datetime
from tqdm import tqdm

### 2. Loading dataframe

In [33]:
ticker_symbol = "RTX"
merged_substituted_df = pd.read_pickle(f"../data/processed/{ticker_symbol}_merged_substituted_df.pkl")

In [34]:
merged_substituted_df

,Open,High,Low,Close,Adj Close,Volume,Ticker Symbol,Publish Date,Title,Body Text,URL,Negative Score,Neutral Score,Positive Score,Compound Score
Date,,,,,,,,,,,,,,,
2017-06-23,76.784142,77.054749,76.456894,76.916298,65.686562,6119398,RTX,2017-06-23,Layoffs planned at Carrier By Seeking Alpha,Please try another search,https://www.investing.com/news/stock-market-ne...,0.000,0.566,0.434,0.3182
2017-06-26,77.035873,77.463814,76.658279,76.696037,65.498444,2613111,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2017-06-27,76.727501,76.865952,76.469475,76.519821,65.347954,2706226,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2017-06-28,76.821899,77.155441,76.582756,77.092514,65.837029,2862107,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2017-06-29,77.042168,77.067337,75.909378,76.557587,65.380203,3747815,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-15,80.510002,81.250000,79.779999,80.629997,80.040001,10757800,RTX,2023-11-15,RTX's Pratt & Whitney Wins $870M Contract for ...,"Published Nov 15, 2023 09:05AM ET\n\nRTX's (RT...",https://www.investing.com/news/assorted/rtxs-p...,0.007,0.879,0.114,0.9571
2023-11-16,80.470001,80.830002,79.930000,80.230003,80.230003,7578700,RTX,2023-11-16,Lockheed Martin shares edge up amid broader ma...,"Published Nov 15, 2023 10:45PM ET\n\nLMT +0.39...",https://www.investing.com/news/stock-market-ne...,0.018,0.848,0.134,0.9887
2023-11-17,80.639999,80.680000,79.419998,79.680000,79.680000,7343900,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000


### 3. Feature Engineering

In [35]:
# Let's add a new column for the next day's adj close price
merged_substituted_labeled_df = merged_substituted_df.copy()
merged_substituted_labeled_df['Next Adj Close'] = merged_substituted_labeled_df['Adj Close'].shift(-1)
merged_substituted_labeled_df.insert(5, 'Next Adj Close', merged_substituted_labeled_df.pop('Next Adj Close'))

In [36]:
merged_substituted_labeled_df[['Adj Close', 'Next Adj Close']]

,Adj Close,Next Adj Close
Date,,
2017-06-23,65.686562,65.498444
2017-06-26,65.498444,65.347954
2017-06-27,65.347954,65.837029
2017-06-28,65.837029,65.380203
2017-06-29,65.380203,65.627426
...,...,...
2023-11-15,80.040001,80.230003
2023-11-16,80.230003,79.680000
2023-11-17,79.680000,79.169998


In [37]:
# Drop the last row that has NaN value
merged_substituted_labeled_df = merged_substituted_labeled_df.iloc[:-1]

In [38]:
merged_substituted_labeled_df

,Open,High,Low,Close,Adj Close,Next Adj Close,Volume,Ticker Symbol,Publish Date,Title,Body Text,URL,Negative Score,Neutral Score,Positive Score,Compound Score
Date,,,,,,,,,,,,,,,,
2017-06-23,76.784142,77.054749,76.456894,76.916298,65.686562,65.498444,6119398,RTX,2017-06-23,Layoffs planned at Carrier By Seeking Alpha,Please try another search,https://www.investing.com/news/stock-market-ne...,0.000,0.566,0.434,0.3182
2017-06-26,77.035873,77.463814,76.658279,76.696037,65.498444,65.347954,2613111,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2017-06-27,76.727501,76.865952,76.469475,76.519821,65.347954,65.837029,2706226,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2017-06-28,76.821899,77.155441,76.582756,77.092514,65.837029,65.380203,2862107,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2017-06-29,77.042168,77.067337,75.909378,76.557587,65.380203,65.627426,3747815,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-14,82.180000,82.250000,80.610001,80.690002,80.099564,80.040001,13128900,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2023-11-15,80.510002,81.250000,79.779999,80.629997,80.040001,80.230003,10757800,RTX,2023-11-15,RTX's Pratt & Whitney Wins $870M Contract for ...,"Published Nov 15, 2023 09:05AM ET\n\nRTX's (RT...",https://www.investing.com/news/assorted/rtxs-p...,0.007,0.879,0.114,0.9571
2023-11-16,80.470001,80.830002,79.930000,80.230003,80.230003,79.680000,7578700,RTX,2023-11-16,Lockheed Martin shares edge up amid broader ma...,"Published Nov 15, 2023 10:45PM ET\n\nLMT +0.39...",https://www.investing.com/news/stock-market-ne...,0.018,0.848,0.134,0.9887


In [39]:
# Add a label for whether the Next Adj Close increased or decreased:
# 1 when it increased or stayed the same.
# 0 when it decreased.

merged_substituted_labeled_df['Price Change Label'] = merged_substituted_labeled_df.apply(lambda x: 1 if (x['Next Adj Close'] >= x['Adj Close']) else 0, axis=1)
merged_substituted_labeled_df.insert(6, 'Price Change Label', merged_substituted_labeled_df.pop('Price Change Label'))

In [40]:
merged_substituted_labeled_df

,Open,High,Low,Close,Adj Close,Next Adj Close,Price Change Label,Volume,Ticker Symbol,Publish Date,Title,Body Text,URL,Negative Score,Neutral Score,Positive Score,Compound Score
Date,,,,,,,,,,,,,,,,,
2017-06-23,76.784142,77.054749,76.456894,76.916298,65.686562,65.498444,0,6119398,RTX,2017-06-23,Layoffs planned at Carrier By Seeking Alpha,Please try another search,https://www.investing.com/news/stock-market-ne...,0.000,0.566,0.434,0.3182
2017-06-26,77.035873,77.463814,76.658279,76.696037,65.498444,65.347954,0,2613111,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2017-06-27,76.727501,76.865952,76.469475,76.519821,65.347954,65.837029,1,2706226,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2017-06-28,76.821899,77.155441,76.582756,77.092514,65.837029,65.380203,0,2862107,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2017-06-29,77.042168,77.067337,75.909378,76.557587,65.380203,65.627426,1,3747815,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-14,82.180000,82.250000,80.610001,80.690002,80.099564,80.040001,0,13128900,RTX,NaN,NaN,NaN,NaN,0.000,0.000,0.000,0.0000
2023-11-15,80.510002,81.250000,79.779999,80.629997,80.040001,80.230003,1,10757800,RTX,2023-11-15,RTX's Pratt & Whitney Wins $870M Contract for ...,"Published Nov 15, 2023 09:05AM ET\n\nRTX's (RT...",https://www.investing.com/news/assorted/rtxs-p...,0.007,0.879,0.114,0.9571
2023-11-16,80.470001,80.830002,79.930000,80.230003,80.230003,79.680000,0,7578700,RTX,2023-11-16,Lockheed Martin shares edge up amid broader ma...,"Published Nov 15, 2023 10:45PM ET\n\nLMT +0.39...",https://www.investing.com/news/stock-market-ne...,0.018,0.848,0.134,0.9887


### 4. Save dataframe

In [41]:
merged_substituted_labeled_df.to_pickle(f"../data/processed/{ticker_symbol}_merged_substituted_labeled_df.pkl")
merged_substituted_labeled_df.to_csv(f"../data/processed/{ticker_symbol}_merged_substituted_labeled_df.csv", sep=',', encoding='utf-8', header=True)